In [1]:
###########################  Reflection  ##################################3
#  Current pipeline is using all the helper functions initially provided. I also implemented a few fuctions like load_image() and 
#save_image().
#Could not complete the line averaging part. I will work on it a bit more. 

In [2]:
#importing some useful packages
from os import listdir
from os.path import isfile, join
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
import math
%matplotlib inline



In [3]:
def load_image(image_path):
    """load the image from the given path."""
    return mpimg.imread(image_path)

In [4]:
def save_image(dir_name, name, img):
    """save the image in the directory"""
    #cv2.imwrite(join(dir_name, name), img)
    plt.imsave(join(dir_name, name), img)

In [5]:
def grayscale(img):
    """Applies the Grayscale transform
    This will return an image with only one color channel
    but NOTE: to see the returned image as grayscale
    you should call plt.imshow(gray, cmap='gray')"""
    return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

In [6]:
def canny(img, low_threshold, high_threshold):
    """Applies the Canny transform"""
    return cv2.Canny(img, low_threshold, high_threshold)


In [7]:
def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)


In [8]:
def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image


In [9]:
def draw_lines(img, lines, color=[255, 0, 0], thickness=9):
    """
    NOTE: this is the function you might want to use as a starting point once you want to 
    average/extrapolate the line segments you detect to map out the full
    extent of the lane (going from the result shown in raw-lines-example.mp4
    to that shown in P1_example.mp4).  
    
    Think about things like separating line segments by their 
    slope ((y2-y1)/(x2-x1)) to decide which segments are part of the left
    line vs. the right line.  Then, you can average the position of each of 
    the lines and extrapolate to the top and bottom of the lane.
    
    This function draws `lines` with `color` and `thickness`.    
    Lines are drawn on the image inplace (mutates the image).
    If you want to make the lines semi-transparent, think about combining
    this function with the weighted_img() function below
    """
#    print ("its a new image")
    for line in lines:
        for x1,y1,x2,y2 in line:
            cv2.line(img, (x1, y1), (x2, y2), color, thickness)
#    left_slope = []
#    right_slope = []
#    for line in lines:
#        for x1,y1,x2,y2 in line:
#            m,b = np.polyfit(np.array([x1,x2]),np.array([y1,y2]),1)
#            if m < 0:
#                left_slope.append(m)
#            elif m > 0:
#                right_slope.append(m)
            
#            print (m,b,line)
#    print ("This is the end")
#    print (sum(right_slope)/(len(right_slope)+1))
#    print (sum(left_slope)/(len(left_slope)+1))
            


In [10]:
def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    """
    `img` should be the output of a Canny transform.
        
    Returns an image with hough lines drawn.
    """
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((*img.shape, 3), dtype=np.uint8)
    draw_lines(line_img, lines)
    return line_img


In [11]:
# Python 3 has support for cool math symbols.

def weighted_img(edges,img, initial_img, α=0.8, β=1., λ=0.):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.
    
    `initial_img` should be the image before any processing.
    
    The result image is computed as follows:
    
    initial_img * α + img * β + λ
    NOTE: initial_img and img must be the same shape!
    """
    color_edges = np.dstack((edges, edges, edges))

    return cv2.addWeighted(initial_img, α, img, β, λ)

In [12]:
def process_img(image):
    image = load_image(image)
    gray_image = grayscale(image)
    blur_gray = gaussian_blur(gray_image, 5)
    edges = canny(blur_gray, 50, 150)#50,150

    imshape = image.shape
    vertices = np.array([[(80,imshape[0]),(450, 310),(imshape[1]-470, 310), (imshape[1]-70,imshape[0])]], dtype=np.int32)
    masked_edges = region_of_interest(edges, vertices)
    
    lines = hough_lines(masked_edges, 1, np.pi/180, 30, 100, 160)
    final_image = weighted_img(edges,image,lines)
    
    return final_image

In [13]:
def process_image(image):
    
    gray_image = grayscale(image)
    blur_gray = gaussian_blur(gray_image, 5)
    edges = canny(blur_gray, 50, 150)

    imshape = image.shape
    vertices = np.array([[(80,imshape[0]),(450, 310),(imshape[1]-470, 310), (imshape[1]-70,imshape[0])]], dtype=np.int32)
    masked_edges = region_of_interest(edges, vertices)
    
    lines = hough_lines(masked_edges, 1, np.pi/180, 30, 100, 160)
    final_image = weighted_img(edges,image,lines)
    
    return final_image

In [14]:
def process_images(input_dir,output_dir):
    names = listdir(input_dir)
    for name in names:
        final = process_img(input_dir+name) 
        save_image(output_dir, "final_"+name, final)

In [15]:
process_images("test_images/","test_images_output")

In [16]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [17]:
white_output = 'test_videos_output/solidWhiteRight.mp4'
clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4")
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

t:   2%|▏         | 5/221 [00:00<00:04, 47.63it/s, now=None]

Moviepy - Building video test_videos_output/solidWhiteRight.mp4.
Moviepy - Writing video test_videos_output/solidWhiteRight.mp4



Moviepy - Done !
Moviepy - video ready test_videos_output/solidWhiteRight.mp4
CPU times: user 3.38 s, sys: 647 ms, total: 4.02 s
Wall time: 7.46 s


In [18]:
yellow_output = 'test_videos_output/solidYellowLeft.mp4'
clip2 = VideoFileClip("test_videos/solidYellowLeft.mp4")
yellow_clip = clip2.fl_image(process_image) #NOTE: this function expects color images!!
%time yellow_clip.write_videofile(yellow_output, audio=False)

t:   1%|          | 7/681 [00:00<00:09, 68.92it/s, now=None]

Moviepy - Building video test_videos_output/solidYellowLeft.mp4.
Moviepy - Writing video test_videos_output/solidYellowLeft.mp4



Moviepy - Done !
Moviepy - video ready test_videos_output/solidYellowLeft.mp4
CPU times: user 10.8 s, sys: 2.01 s, total: 12.9 s
Wall time: 23.8 s
